In [59]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\user\Downloads\Bank_Personal_Loan_Modelling.csv')
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [60]:
df = df.drop(columns = ["ID","ZIP Code"])

In [61]:
X = df.drop("Personal Loan",axis=1)
y = df["Personal Loan"]

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [63]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [64]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'

C:\Users\user\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [65]:
def fitness(weights):
    w0 = weights[:66].reshape((11, 6))
    w1 = weights[66:72].reshape((6,1))
    b0= np.array([0.,0.,0.,0.,0.,0.])
    b1 = np.array([0.])


    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    model.compile(optimizer=optimizer, loss=loss)
    Loss = model.evaluate(X_train, y_train, verbose=0)
    return -Loss

In [66]:
def genetic_algorithm(population_size, generations, mutation_rate):
    # Initialize the population and belief space with random weights
    population = [np.random.uniform(low=-1, high=1, size=77) for _ in range(population_size)]
    
    for generation in range(generations):
        fitness_scores = [fitness(x) for x in population]

        indices = np.argsort(fitness_scores)[-2:]
        parents = [population[i] for i in indices]

        offspring = []
        for _ in range(population_size - len(parents)):
            parent1 = parents[0].flatten()
            parent2 = parents[1].flatten()

            #Crossover
            crossover_point = np.random.randint(0, len(parent1))
            child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child = child.reshape(parents[0].shape)

            #Mutation
            for i in range(len(child)):
                if np.random.uniform() < mutation_rate:
                    child[i] += np.random.normal(loc=0, scale=0.1)

            offspring.append(child)

        # Combine parents and offspring to form the new population
        population = parents + offspring
        #
        print("Finished Generation:",(generation+1))

    fitness_scores = [fitness(x) for x in population]
    best_index = np.argmax(fitness_scores)
    best_weights = population[best_index]

    return best_weights

In [67]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

In [68]:
model.weights

[<tf.Variable 'dense_8/kernel:0' shape=(11, 6) dtype=float32, numpy=
 array([[ 2.34129131e-01,  1.91909552e-01,  3.49326253e-01,
          2.10011005e-02,  2.12186575e-01, -5.68343878e-01],
        [-7.03738332e-02,  2.98021734e-01, -2.97389746e-01,
         -2.90464729e-01,  7.61168599e-02, -4.55164611e-01],
        [-4.28861737e-01, -3.30457419e-01, -3.00170183e-02,
          6.71852231e-02, -3.92816186e-01, -4.61324066e-01],
        [-9.51230526e-02,  1.54421926e-02, -2.09104449e-01,
         -4.89261061e-01, -3.86745870e-01,  9.27250385e-02],
        [ 3.01551044e-01,  3.88672769e-01,  1.00289464e-01,
          3.62105131e-01, -4.82516646e-01, -2.58139402e-01],
        [ 2.54809856e-04,  3.59883487e-01,  5.02012730e-01,
          4.61678982e-01,  2.42800713e-02, -1.56323761e-01],
        [-1.59699231e-01, -5.28729796e-01,  3.09130311e-01,
          4.72736955e-01, -1.93858117e-01, -8.08849335e-02],
        [-2.70882905e-01, -2.33978868e-01, -5.02681315e-01,
         -2.01318830e-01

In [69]:
best_weights = genetic_algorithm(population_size=20, generations=20, mutation_rate=0.1)
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])

model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])

model.compile(optimizer=optimizer, loss=loss)

Finished Generation: 1
Finished Generation: 2
Finished Generation: 3
Finished Generation: 4
Finished Generation: 5
Finished Generation: 6
Finished Generation: 7
Finished Generation: 8
Finished Generation: 9
Finished Generation: 10
Finished Generation: 11
Finished Generation: 12
Finished Generation: 13
Finished Generation: 14
Finished Generation: 15
Finished Generation: 16
Finished Generation: 17
Finished Generation: 18
Finished Generation: 19
Finished Generation: 20


In [70]:
y_pred = model.predict(X_test)
y_pred = (y_pred < 0.00005)

47/47 [==============================] - 0s 716us/step


In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9146666666666666